# Data understanding

De gebruikte dataset is “Vestigingen_Gemeente" (https://opendata.cbs.nl/statline/#/CBS/nl/dataset/81575NED/table?ts=1584707583585). 

De CBS data is een csv bestand, dit staat voor comma seperated values of door comma gescheide waarden in het nederlands.
De data bevat de volgende kolomen:
 - `ID` is de unieke waarde voor elke row.
 - `Bedrijfstakken/branches SBI 2008` geeft de waarde weer in welke bedrijfstak/branch de bedrijfen zit.
 - `Perioden` geeft de periode aan waarin de bedrijfstak/branch actief in is.
 - `Regio's` is de regio waarin de behorende bedrijfstak/branch actief in is.
 - `Vestigingen (aantal)` is de aantal vestigingen die binnen die bedrijfstak/branch geteld zijn per regio en per periode.

## Beschrijven van de data

Hieronder staat een beschrijving van de data vanuit de Centraal Bureau Statistieken. Dit wordt telkens door middel van een stukje code weergegeven. 

In [2]:
import pandas as pd
import geopandas as gpd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 50)

# Dataset inlezen
file = "../../raw-data/Vestegingen_Gemeente.csv"
postcode_file = "../../raw-data/gemeenten-provincie.xlsx"
data_pc = pd.read_excel(postcode_file)
data_vg = pd.read_csv(file, sep=";")

# Dataverkennen door dataframe te openen
data_vg

FileNotFoundError: [Errno 2] No such file or directory: '../../raw-data/gemeenten-provincie.xlsx'

In [3]:
test = data_vg["Regio's"].unique()
test

NameError: name 'data_vg' is not defined

In [4]:
data_pc

NameError: name 'data_pc' is not defined

In [5]:
# Info over de kolommen
groepeerde_data_vg = data_vg.groupby(["Regio's", "Perioden"]).sum()
info_kolommen_vg = data_vg.info()
info_kolommen_vg

NameError: name 'data_vg' is not defined

In [6]:
# Beschrijvende statistieken van de data
beschrijvende_stats_vg = data_vg.describe()
beschrijvende_stats_vg

NameError: name 'data_vg' is not defined

In [7]:
# Informatie over de kolomnamen in de Dataframe
data_vg.columns

NameError: name 'data_vg' is not defined

In [8]:
# Controleren op nulwaarden in de dataframe
data_vg.isna().sum()

NameError: name 'data_vg' is not defined

## Data exploratie

In [9]:
# Het laten zien van hoevaak een regio voorkomt in de dataset.
sb.countplot(y="Regio's", data=data_vg)

NameError: name 'data_vg' is not defined

In [10]:
g = sb.catplot(x="Regio's", y="Vestigingen (aantal)",
                data=data_vg, saturation=10,
                kind="bar", ci=None, aspect=15)
(g.set_axis_labels("", "Vestigingen (aantal)")
  .set_titles("{col_name} {col_var}")
  .despine(left=True))

NameError: name 'data_vg' is not defined

In [11]:
sb.pairplot(data_vg, kind="reg")

NameError: name 'data_vg' is not defined

In [12]:
sb.heatmap(data_vg.corr(), annot=True)

NameError: name 'data_vg' is not defined

### Samenvatting

Er zijn nu 2 datasets:
- `data_vg` is de 'standaard' dataframe met alle kolommen en waardes die compleet zijn.
- `groepeerde_data_vg` is de **gegroepeerde** dataset, gegroepeerd op de kolommen `Regio's` en `Perioden`.

# Data Preparation

Tijdens de analyse van de benodigde data worden de datasets `data_vg` en `groepeerde_data_vg` gebruikt. Van deze dataset wordt de data vanaf het jaar 2013 t/m 2018 gebruikt. De jaren 2019 en 2020 zijn wel beschikbaar, maar nog niet volledig.

Om te bepalen welke data gebruikt dient te worden, wordt er gekeken naar hoe de data aan de data mining goals kan voldoen. De data mining goal is: **Het verhogen van succesvolle woningzoekers die op zoek zijn naar een woonplek**.

De `data_vg` focust zich op het zoeken van mogelijke werkplekken van een woningzoekende in de ideale regio. De user story die beantwoordt dient te worden met de `data_vg` is **Als bezoeker wil ik graag dat werkgelegenheid mee wordt geteld in de resultaten, zodat ik kan de hoeveelheid mogelijke werkplekken per regio als resultaat zie**.

De meest recente data van `data_vg` wordt gebruikt om deze user story te beantwoorden. Deze nieuwe dataframe zal `data_vg_2018` heten.

Voor de volledigheid wordt voor elke periode een eigen dataframe gemaakt, in de syntax van de dataframe wordt dit weergegeven door `data_vg_[jaartal]`.

Ook wordt er een dataframe `data_pc` toegevoegd aan `data_vg`, zodat per regio de plaats, gemeente en provincie wordt weergegeven.

In [13]:
# Een dataframe per Periode
data_vg_2013 = data_vg.loc[data_vg["Perioden"] == 2013]
data_vg_2014 = data_vg.loc[data_vg["Perioden"] == 2014]
data_vg_2015 = data_vg.loc[data_vg["Perioden"] == 2015]
data_vg_2016 = data_vg.loc[data_vg["Perioden"] == 2016]
data_vg_2017 = data_vg.loc[data_vg["Perioden"] == 2017]
data_vg_2018 = data_vg.loc[data_vg["Perioden"] == 2018]

NameError: name 'data_vg' is not defined

## Select Data

Van `data_vg_2018` worden alle kolommen gebruikt voor het beantwoorden van de user story.

Van `data_pc` wordt een aantal kolommen gebruikt, namelijk:
- `Plaats`.
- `Gemeente`.
- `Provincie`.



In [14]:
data_pc.columns

NameError: name 'data_pc' is not defined

In [15]:
data_pc

NameError: name 'data_pc' is not defined

## Clean Data

In dit onderdeel wordt gekeken welke acties moeten worden ondernomen om de data op te schonen. Er wordt bijvoorbeeld gekeken naar rare waardes, aanpassingen in de records of overige acties.

In [16]:
# Unieke waarden in de verschillende kolommen van data_vg
test = ["Bedrijfstakken/branches SBI 2008", "Perioden", "Regio's", "Vestigingen (aantal)"]
for i in test:
    print(i, data_vg[i].unique())

NameError: name 'data_vg' is not defined

Uit de vorige functie is te herlijden dat:
- `Bedrijfstakken/branches SBI 2008` heeft geen opmerkelijke waarden. Alle records zijn volgens hetzelfde format ingevuld.
- `Perioden` is gereed voor gebruik.
- `Regio's` heeft een aantal dubbele waarden, maar die worden weer gescheiden door de afkorting van de province van de Regio.
- `Vestigingen (aantal)` heeft een opmerkelijke notatie die op basis van vorige functie anders word weergegeven dan in de tabelweergave. De waarden zijn echter correct en hetzelfde, maar met een andere notatie. Het valt op dat er in deze kolom een NaN waarde zit, dat dient aangepast te worden.

In [17]:
# Controleren op de lege waardes
data_vg_2018.isna().sum()

NameError: name 'data_vg_2018' is not defined

In [18]:
# Invullen van de Na/NaN waarden in kolom Vestigingen (aantal) door 0
data_vg2 = data_vg["Vestigingen (aantal)"].fillna(0)
data_vg2

NameError: name 'data_vg' is not defined

## Construct data

Om de data bruikbaar te maken voor user input in het formulier, wordt een classificatie aangebracht aan het aantal vestigingen. De waarde van deze classificatie zal bepaald worden door de uitkomst van data_vg.describe() uit de data understanding. De klassen zijn:
- `Laag`.
- `Middel`.
- `Hoog`.

Er wordt nog gewerkt aan een klasse `Geen` om de sectoren met 0 vestigingen te onderscheiden van de klasse `Laag`.

In [19]:
data_vg_2018["Bedrijfstakken/branches SBI 2008"].unique()

NameError: name 'data_vg_2018' is not defined

In [20]:
data_vg_2018

NameError: name 'data_vg_2018' is not defined

In [21]:
# Samenvoegen van data_vg en data_pc2
data_vg_2018v2 = pd.merge(data_vg_2018, data_pc, left_on="Regio's", right_on="Regio's").drop_duplicates().reset_index(drop=True)
data_vg_2018v2

NameError: name 'data_vg_2018' is not defined

De data van data_vg_2018 wordt gecategoriseerd op een `Laag`, `Middel` of `Hoog` aantal vestigingen in een bepaalde branche.

In [22]:
#data_vg_2018v2.drop(columns=["Klasse vestigingsaantal"])
vestigingen = data_vg_2018v2["Vestigingen (aantal)"]


#m1 = df['maker_nm'].str.contains("Sam|Mike")
#m2 = df['maker_nm'].str.contains("Andy|John|Paul|Jay")

#df['maker_grp'] = np.select([m1,m2], ['Class1','Class2'], default='Class3')

# Gebruik de aantallen die bij vg_data.describe() staan
m1 = vestigingen < 200
m2 = vestigingen > 1000

data_vg_2018v2['Klasse vestigingsaantal'] = np.select([m1,m2], ['Laag','Hoog'], default='Middel')

NameError: name 'data_vg_2018v2' is not defined

In [23]:
data_vg_2018v2

NameError: name 'data_vg_2018v2' is not defined

# Uitkomst Analyse

Uit de analyse van deze notebook, wordt gekeken naar de requirements uit de User Input die de user story nodig heeft. Hieruit wordt geadviseerd welke opties de user moet kunnen invoeren in het formulier.

Vervolgens wordt er een advies gegeven wat voor beschrijvende statistieken en voorspellende aspecten uit de dataset toegepast kunnen worden, zodat ook die bijbehorende user stories uitgevoerd kunnen worden op de huidige dataset.

## User input in het formulier
De huidige user story heeft als resultaat een aantal voorstellen voor verwerking in het formulier. De user story is:
- **Als bezoeker wil ik graag dat werkgelegenheid mee wordt geteld in de resultaten, zodat ik kan de hoeveelheid mogelijke werkplekken per regio als resultaat zie**.

De dataset werkt met een classificatie van het aantal vestigingen per Regio per Bedrijfsbranche. Het aantal vestigingen zijn mogelijke werkplekken waar de user zou kunnen werken.

### Advies Formulier
In het formulier is het belangrijk dat de user een voorkeur voor bedrijfsbranche kan aanvinken. Dit zou bijvoorbeerld een lijstje met checkboxes kunnen zijn, zodat de user eventueel meerdere branches kan selecteren. Als extra optie kan de user aangeven welke klasse aan mogelijkheden het best bij de user past in bijvoorbeeld een drop down menu. Sommige gebruikers willen graag zo veel mogelijk opties zien, terwijl er ook gebruikers zijn die het fijner vinden om een geringe aantal opties te zien verschijnen.

### Advies Beschrijvende statistieken en voorspellende aspecten
De data in de analyse is gebaseerd op de periode 2018. Er is veel meer data beschikbaar van de perioden voor 2018 en een nog niet volledig overzicht van de jaren 2019 en 2020.

Voor het uitwerken van user stories over het beschrijvende gedeelte van deze dataset, kan gekeken worden naar de perioden voor 2018 en hier beschrijvende statistieken op uitvoeren. Er zou bijvoorbeeld een mooi overzicht van de ontwikkelingen van het aantal vestigingen van een bepaalde branche in een regio als overzicht gegeven kunnen worden.

Op basis van deze historische data kan ook een invulling gegeven worden aan het voorspellende aspect van de werkgelegenheid.